In [ ]:
!pip install ../input/weights/timm-0.3.1-py3-none-any.whl

In [ ]:
import random
import time
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import functional as FT
from PIL import Image,  ImageChops
import cv2

from sklearn.metrics import cohen_kappa_score
import timm

device = "cuda:0"


In [ ]:
import torch
import torch.nn as nn
import torchvision
import csv
import timm
import time
import glob
import copy
import os
import json
import cv2
import numpy as np
import pickle
import torch.optim as optim
import torch
import torch.nn.functional as F
from torch import nn as nn
from torch.optim import lr_scheduler
from timm.models import *
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, models, datasets
from PIL import Image
from PIL import Image, ImageEnhance, ImageOps
from torchvision.models._utils import IntermediateLayerGetter
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops.feature_pyramid_network import (
    FeaturePyramidNetwork,
    LastLevelMaxPool,
)
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.ops.misc import FrozenBatchNorm2d
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import misc as misc_nn_ops
from torch import Tensor, Size
from torch.jit.annotations import List, Optional, Tuple
from torch.nn.parameter import Parameter


def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, flatten=False):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
        self.flatten = flatten
    def forward(self, x):
        x = gem(x, p=self.p, eps=self.eps)
        if self.flatten:
            x = x.flatten(1)
        return x
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'




class FrozenBatchNorm2d(torch.nn.Module):
    """
    BatchNorm2d where the batch statistics and the affine parameters
    are fixed
    """

    def __init__(
        self,
        num_features: int,
        eps: float = 1e-5,
        n: Optional[int] = None,
    ):
        # n=None for backward-compatibility
        if n is not None:
            warnings.warn("`n` argument is deprecated and has been renamed `num_features`",
                          DeprecationWarning)
            num_features = n
        super(FrozenBatchNorm2d, self).__init__()
        self.eps = eps
        self.register_buffer("weight", torch.ones(num_features))
        self.register_buffer("bias", torch.zeros(num_features))
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def _load_from_state_dict(
        self,
        state_dict: dict,
        prefix: str,
        local_metadata: dict,
        strict: bool,
        missing_keys: List[str],
        unexpected_keys: List[str],
        error_msgs: List[str],
    ):
        num_batches_tracked_key = prefix + 'num_batches_tracked'
        if num_batches_tracked_key in state_dict:
            del state_dict[num_batches_tracked_key]

        super(FrozenBatchNorm2d, self)._load_from_state_dict(
            state_dict, prefix, local_metadata, strict,
            missing_keys, unexpected_keys, error_msgs)

    def forward(self, x: Tensor) -> Tensor:
        # move reshapes to the beginning
        # to make it fuser-friendly
        w = self.weight.reshape(1, -1, 1, 1)
        b = self.bias.reshape(1, -1, 1, 1)
        rv = self.running_var.reshape(1, -1, 1, 1)
        rm = self.running_mean.reshape(1, -1, 1, 1)
        scale = w * (rv + self.eps).rsqrt()
        bias = b - rm * scale
        return x * scale + bias

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.weight.shape[0]}, eps={self.eps})"
    

class Linear(nn.Linear):
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        if torch.jit.is_scripting():
            bias = self.bias.to(dtype=input.dtype) if self.bias is not None else None
            return F.linear(input, self.weight.to(dtype=input.dtype), bias=bias)
        else:
            return F.linear(input, self.weight, self.bias)
    
    
    
    
class backboneNet_efficient(nn.Module):
    def __init__(self):
        super(backboneNet_efficient, self).__init__()
#         norm_layer=FrozenBatchNorm2d
        net = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)       
        layers_to_train = ['blocks']       
        for name, parameter in net.named_parameters():
            if all([not name.startswith(layer) for layer in layers_to_train]):
                parameter.requires_grad_(False)
#         self.num_features = 1408
        self.num_features = 1792
        self.conv_stem = net.conv_stem
        self.bn1 = net.bn1
        self.act1 = net.act1       
        self.block0 = net.blocks[0]
        self.block1 = net.blocks[1]
        self.block2 = net.blocks[2]
        self.block3 = net.blocks[3]
        self.block4 = net.blocks[4]
        self.block5 = net.blocks[5]
        self.block6 = net.blocks[6]       
        self.conv_head = net.conv_head
        self.bn2 = net.bn2
        self.act2 = net.act2
        self.global_pool =  net.global_pool
        self.drop_rate = 0.4
        
#         self.dense1 = Linear(self.num_features, self.num_features, bias=True)
        
        self.rg_cls = Linear(self.num_features, 1, bias=True)
        self.cls_cls = Linear(self.num_features, 5, bias=True)
        self.ord_cls = Linear(self.num_features, 4, bias=True)
        
    def forward(self, x):
        
        x1 = self.conv_stem(x)
        x2 = self.bn1(x1)
        x3 = self.act1(x2)
        x4 = self.block0(x1)
        x5 = self.block1(x4)
        x6 = self.block2(x5)
        x7 = self.block3(x6)
        x8 = self.block4(x7)
        x9 = self.block5(x8)
        x10 = self.block6(x9)       
        x11 = self.conv_head(x10)
        x12 = self.bn2(x11)
        x13 = self.act2(x12)
        x14  = self.global_pool(x13)
        if self.drop_rate > 0.:
            x14 = F.dropout(x14, p=self.drop_rate, training=self.training)
            
#         x15 = self.dense1(x14)
#         x15 = self.act2(x15)
        
#         x16 = self.dense1(x14)
#         x16 = self.act2(x16)
        
#         x17 = self.dense1(x14)
#         x17 = self.act2(x17)
            
        x15 = self.rg_cls(x14)
        x16 = self.cls_cls(x14)
        x17 = self.ord_cls(x14)
       
        return x15, x16, x17
    
#     'test_Cpixel': transforms.Compose([
#         transforms.Resize((280,280)),
#         transforms.CenterCrop(256),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
    

In [ ]:
# utils
threshold = [0.75, 1.5, 2.5, 3.5]
def regress2class(out):
    prediction = 0
    
    for i in range(4):
        prediction += (out.data >= threshold[i]).squeeze().cpu().item()
    
    return prediction

def ordinal2class_prob(out):
    pred_prob = torch.zeros(out.size(0), 5).cuda()
    pred_prob[:, 0] = (1 - out[:, 0]).squeeze()
    pred_prob[:, 1] = (out[:, 0] * (1 - out[:, 1])).squeeze()
    pred_prob[:, 2] = (out[:, 1] * (1 - out[:, 2])).squeeze()
    pred_prob[:, 3] = (out[:, 2] * (1 - out[:, 3])).squeeze()
    pred_prob[:, 4] = out[:, 3].squeeze().cpu()
    
    return F.softmax(pred_prob, dim=1)


def regress2class_prob(out):
    pred_prob = torch.zeros((out.size(0), 5)).cuda()

    for i in range(out.size(0)):
        if out[i] < 4.:
            l1 = int(math.floor(out[i]))
            l2 = int(math.ceil(out[i]))
            pred_prob[i][l1] = 1 - (out[i] - l1)
            pred_prob[i][l2] = 1 - (l2 - out[i])
        else:
            pred_prob[i][4] = 1.
    
    return pred_prob

def combine3output(r_out, c_out, o_out):
    R = regress2class(r_out.data)
    _, C = torch.max(c_out.data, 1)
    C = C.squeeze().item()
    _, O = torch.max(o_out.data, 1)
    O = O.squeeze().item()
    
    P = (R + C + O) / 3.
    P = int(round(P))
    
    return P

In [ ]:
# models
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, flatten=False):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
        self.flatten = flatten
    def forward(self, x):
        x = gem(x, p=self.p, eps=self.eps)
        if self.flatten:
            x = x.flatten(1)
        return x
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'


class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()

        self.backbone = timm.models.tf_efficientnet_b5_ns(pretrained=False)
        self.backbone.global_pool = GeM(flatten=True)
        self.regressor = nn.Linear(1000, 1)

    def forward(self, x):
        x = self.backbone(x)
        out = self.regressor(x)
        out = torch.sigmoid(out) * 4.5
        
        return out


class ThreeStage_Model(nn.Module):
    def __init__(self, backbone=None):
        super(ThreeStage_Model, self).__init__()
        
        self.backbone = timm.models.tf_efficientnet_b4_ns(pretrained=False)
        self.backbone.global_pool = GeM(flatten=True)

        self.classifier = nn.Sequential(
                nn.SiLU(),
                nn.Linear(1000, 500),
                nn.SiLU(),
                nn.Linear(500, 5),
            )
        
        self.regressor = nn.Sequential(
                nn.SiLU(),
                nn.Linear(1000, 500),
                nn.SiLU(),
                nn.Linear(500, 1),
            )

        self.ordinal = nn.Sequential(
                nn.SiLU(),
                nn.Linear(1000, 500),
                nn.SiLU(),
                nn.Linear(500, 4),
            )
        
        self.final_regressor = nn.Sequential(
                nn.SiLU(),
                nn.Linear(10, 1),
            )

    def forward(self, x, final=False):
        x = self.backbone(x)
        
        c_out = self.classifier(x)
        r_out = self.regressor(x)
        o_out = self.ordinal(x)
        
        if final:
            out = torch.cat((c_out, r_out, o_out), 1)
            out = self.final_regressor(out)
            out = torch.sigmoid(out) * 4.5
            return out
        else:
            r_out = torch.sigmoid(r_out) * 4.5
            o_out = torch.sigmoid(o_out)
            return c_out, r_out, o_out

In [ ]:
# image transformation
class photometric_distort(object):
    def __call__(self, image):
        distortions = [FT.adjust_brightness,
                       FT.adjust_contrast,
                       FT.adjust_saturation,
                       FT.adjust_hue]

        random.shuffle(distortions)

        for d in distortions:
            if random.random() < 0.5:
                if d.__name__ is 'adjust_hue':
                    adjust_factor = random.uniform(-16 / 255., 16 / 255.)
                else:
                    adjust_factor = random.uniform(0.7, 1.3)
                # Apply this distortion
                image = d(image, adjust_factor)

        return image


class cropTo4_3(object):
    def __call__(self, image):
        w, h = image.size
        
        if (w / h) >= (4 / 3):
            new_h = h
            new_w = int(h * 4 / 3)
        else:
            new_h = int(w * 3 / 4)
            new_w = w
            
        left = (w - new_w)/2
        top = (h - new_h)/2
        right = left + new_w
        bottom = top + new_h

        return image.crop((left, top, right, bottom))

    
class trim(object):
    def __call__(self, image):
        bg = Image.new(image.mode, image.size, image.getpixel((0,0)))
        diff = ImageChops.difference(image, bg)
        diff = ImageChops.add(diff, diff, 2.0, -10)
        bbox = diff.getbbox()
        if bbox:
             return image.crop(bbox)
            
            
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
test_ids = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
test_ids = np.squeeze(test_ids.values)

transform1 = transforms.Compose([
                trim(),
                cropTo4_3(),
                transforms.Resize((288, 384)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.384, 0.258, 0.174], std=[0.124, 0.089, 0.094]),
            ])

transform2 = transforms.Compose([
                transforms.Resize((280,280)),
                transforms.CenterCrop(256),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])


net1 = ThreeStage_Model()
net1.load_state_dict(torch.load("../input/weights/0.926_B4_3stage_5epoch_320finetune.pkl"))
net1 = net1.to(device)
net1.eval()

net2 = backboneNet_efficient()
net2.load_state_dict(torch.load("../input/weights/1.pth", map_location='cpu'))
net2 = net2.to(device)
net2.eval()

net3 = backboneNet_efficient()
net3.load_state_dict(torch.load("../input/weights/2.pth", map_location='cpu'))
net3 = net3.to(device)
net3.eval()

# net4 = backboneNet_efficient()
# net4.load_state_dict(torch.load("../input/weights/3.pth", map_location='cpu'))
# net4 = net4.to(device)
# net4.eval()

net5 = backboneNet_efficient()
net5.load_state_dict(torch.load("../input/weights/0.929292_4", map_location='cpu'))
net5 = net5.to(device)
net5.eval()

In [ ]:
submission = []
thrs = [0.7, 1.5, 2.5, 3.5]
with torch.no_grad():
    for i, idx in enumerate(test_ids):
        print(i)
        image_name = "../input/aptos2019-blindness-detection/test_images/{}.png".format(idx)
        img1 = Image.open(image_name).convert('RGB')
        img1 = transform1(img1).unsqueeze(0).to(device)
        
        img2 = cv2.imread(image_name)
        img2 = crop_image_from_gray(img2)
        img2 = Image.fromarray(cv2.cvtColor(img2,cv2.COLOR_BGR2RGB))
        img2 = transform2(img2).unsqueeze(0).to(device)
        
        # model 1
        _, r_out, _ = net1(img1)
        pred1 = int(regress2class(r_out.data.squeeze(1)))
        
        # model 2
        rg_outputs, cls_outputs, ord_outputs = net2(img2)
        rg_outputs = torch.sigmoid(rg_outputs) * 4.5
        
        cls_softmax = torch.nn.Softmax(dim=1)
        cls_outputs = cls_softmax(cls_outputs)
        _, cls_preds = torch.max(cls_outputs, 1)
        cls_predict = cls_preds.cpu().numpy()
        
        ord_outputs = torch.sigmoid(ord_outputs)
        rg_outputs = rg_outputs.unsqueeze(1)                    
        rg_outputs[rg_outputs < thrs[0]] = 0
        rg_outputs[(rg_outputs >= thrs[0]) & (rg_outputs < thrs[1])] = 1
        rg_outputs[(rg_outputs >= thrs[1]) & (rg_outputs < thrs[2])] = 2
        rg_outputs[(rg_outputs >= thrs[2]) & (rg_outputs < thrs[3])] = 3
        rg_outputs[rg_outputs >= thrs[3]] = 4                                           
        rg_preds = rg_outputs.tolist()
        rg_predict = rg_preds[0][0]

        ord_outputs[ord_outputs >= 0.5] = 1
        ord_outputs[ord_outputs < 0.5] = 0
        ord_predict = sum(ord_outputs[0].cpu().numpy())
        
        pred2 = int(round((rg_predict[0] + cls_predict[0] + ord_predict) / 3.))
        
        # model 3
        rg_outputs, cls_outputs, ord_outputs = net3(img2)
        rg_outputs = torch.sigmoid(rg_outputs) * 4.5
        
        cls_softmax = torch.nn.Softmax(dim=1)
        cls_outputs = cls_softmax(cls_outputs)
        _, cls_preds = torch.max(cls_outputs, 1)
        cls_predict = cls_preds.cpu().numpy()
        
        ord_outputs = torch.sigmoid(ord_outputs)
        rg_outputs = rg_outputs.unsqueeze(1)                    
        rg_outputs[rg_outputs < thrs[0]] = 0
        rg_outputs[(rg_outputs >= thrs[0]) & (rg_outputs < thrs[1])] = 1
        rg_outputs[(rg_outputs >= thrs[1]) & (rg_outputs < thrs[2])] = 2
        rg_outputs[(rg_outputs >= thrs[2]) & (rg_outputs < thrs[3])] = 3
        rg_outputs[rg_outputs >= thrs[3]] = 4                                           
        rg_preds = rg_outputs.tolist()
        rg_predict = rg_preds[0][0]

        ord_outputs[ord_outputs >= 0.5] = 1
        ord_outputs[ord_outputs < 0.5] = 0
        ord_predict = sum(ord_outputs[0].cpu().numpy())
        
        pred3 = int(round((rg_predict[0] + cls_predict[0] + ord_predict) / 3.))
        
#         # model 4
#         rg_outputs, cls_outputs, ord_outputs = net4(img2)
#         rg_outputs = torch.sigmoid(rg_outputs) * 4.5
        
#         cls_softmax = torch.nn.Softmax(dim=1)
#         cls_outputs = cls_softmax(cls_outputs)
#         _, cls_preds = torch.max(cls_outputs, 1)
#         cls_predict = cls_preds.cpu().numpy()
        
#         ord_outputs = torch.sigmoid(ord_outputs)
#         rg_outputs = rg_outputs.unsqueeze(1)                    
#         rg_outputs[rg_outputs < thrs[0]] = 0
#         rg_outputs[(rg_outputs >= thrs[0]) & (rg_outputs < thrs[1])] = 1
#         rg_outputs[(rg_outputs >= thrs[1]) & (rg_outputs < thrs[2])] = 2
#         rg_outputs[(rg_outputs >= thrs[2]) & (rg_outputs < thrs[3])] = 3
#         rg_outputs[rg_outputs >= thrs[3]] = 4                                           
#         rg_preds = rg_outputs.tolist()
#         rg_predict = rg_preds[0][0]

#         ord_outputs[ord_outputs >= 0.5] = 1
#         ord_outputs[ord_outputs < 0.5] = 0
#         ord_predict = sum(ord_outputs[0].cpu().numpy())
        
#         pred4 = int(round((rg_predict[0] + cls_predict[0] + ord_predict) / 3.))
        
        # model 5
        rg_outputs, cls_outputs, ord_outputs = net5(img2)
        rg_outputs = torch.sigmoid(rg_outputs) * 4.5
        
        cls_softmax = torch.nn.Softmax(dim=1)
        cls_outputs = cls_softmax(cls_outputs)
        _, cls_preds = torch.max(cls_outputs, 1)
        cls_predict = cls_preds.cpu().numpy()
        
        ord_outputs = torch.sigmoid(ord_outputs)
        rg_outputs = rg_outputs.unsqueeze(1)                    
        rg_outputs[rg_outputs < thrs[0]] = 0
        rg_outputs[(rg_outputs >= thrs[0]) & (rg_outputs < thrs[1])] = 1
        rg_outputs[(rg_outputs >= thrs[1]) & (rg_outputs < thrs[2])] = 2
        rg_outputs[(rg_outputs >= thrs[2]) & (rg_outputs < thrs[3])] = 3
        rg_outputs[rg_outputs >= thrs[3]] = 4                                           
        rg_preds = rg_outputs.tolist()
        rg_predict = rg_preds[0][0]

        ord_outputs[ord_outputs >= 0.5] = 1
        ord_outputs[ord_outputs < 0.5] = 0
        ord_predict = sum(ord_outputs[0].cpu().numpy())
        
        pred5 = int(round((rg_predict[0] + cls_predict[0] + ord_predict) / 3.))
        
#         P = int(round((pred1 + pred2 + pred3 + pred4 + pred5) / 5.))
        P = int(round((pred1 + pred2 + pred3 + pred5) / 4.))
        submission.append([idx, P])

submission = np.array(submission)

In [ ]:
df = pd.DataFrame(submission, columns=["id_code", "diagnosis"])
df.to_csv("submission.csv", index=False)